### Bonus Question

### 1. Sample final states in the statevector or tensor representations


Sampling final states involves applying quantum measurement to the current state of a qubit. This measurement typically projects the state onto the computational basis states, which are \( |0\rangle \) and \( |1\rangle \). The corresponding projectors for these states are:

\begin{align*}
    \text{Projector for } |0\rangle: & \quad P_0 = |0\rangle \langle 0| = \begin{bmatrix} 1 & 0 \\ 0 & 0 \end{bmatrix} \\
    \text{Projector for } |1\rangle: & \quad P_1 = |1\rangle \langle 1| = \begin{bmatrix} 0 & 0 \\ 0 & 1 \end{bmatrix}
\end{align*}

When we measure a qubit, we apply one of these projectors. The measurement outcome (either $ |0\rangle $ or $ |1\rangle $) depends on the probabilities derived from the state's amplitudes. After applying a projector, we normalize the resulting state to reflect the new state of the qubit after measurement.

In [3]:
import numpy as np
from scipy.linalg import norm

# Define projectors for |0⟩ and |1⟩
projectors = [np.array([[1, 0], [0, 0]]), np.array([[0, 0], [0, 1]])]

To determine the outcome of a measurement, we first calculate the probabilities of obtaining each state. For example, for a qubit in the state 

$$|\psi\rangle = \alpha |0\rangle + \beta |1\rangle,$$

the probabilities of measuring $|0\rangle$ and $|1\rangle$ are calculated using the projectors $P_0$ and $P_1$:

$$P(0) = \langle \psi | P_0 | \psi \rangle = |\alpha|^2,$$

$$P(1) = \langle \psi | P_1 | \psi \rangle = |\beta|^2.$$


Once the probabilities are determined, we can measure the qubit. 

In [4]:
# Initialize the state
def initialize_state(n):
    state = np.zeros((2,) * n)
    state[(0,) * n] = 1  # Start with |0...0⟩ state
    return state

# Project state onto |0⟩ or |1⟩ for qubit i
def project(i, j, state):
    projected = np.tensordot(projectors[j], state, (1, i))
    return np.moveaxis(projected, 0, i)

# Example: Hadamard gate
H_matrix = 1/np.sqrt(2) * np.array([[1, 1],
                                    [1, -1]])
def H(i, psi):
    psi = np.tensordot(H_matrix, psi, (1, i))
    psi = np.moveaxis(psi, 0, i)
    return psi

# Measure qubit
def measure(i, psi):
    # Project onto |0⟩ for qubit i and calculate the probability of measuring |0⟩
    projected_0 = project(i, 0, psi)
    prob_0 = norm(projected_0.flatten())**2  # Probability of |0⟩

    # Decide the measurement outcome based on probability
    if np.random.random() < prob_0:
        psi = projected_0 / norm(projected_0)  # Collapse to |0⟩ state for qubit i
        return 0, psi
    else:
        projected_1 = project(i, 1, psi)
        psi = projected_1 / norm(projected_1)  # Collapse to |1⟩ state for qubit i
        return 1, psi

In [21]:
N = 2 # Number of qubits
state = initialize_state(N)  # Initialize a 2-qubit state in |00⟩

# Apply Hadamard gate to both qubits to create superposition
for i in range(N):
    state = H(i, state)

# Measure the first qubit
outcome, state = measure(0, state)
print("Measurement result of qubit-0:", outcome)
print("State after measurement:", state.flatten())


Measurement result of qubit-0: 0
State after measurement: [0.70710678 0.70710678 0.         0.        ]


An example of 100 measurements for this state will result in probabilities close to 0.5:0.5, as shown below:

In [42]:
count_0 = 0 # count of measurement outcome 0
count_1 = 0 # count of measurement outcome 1
N = 3 # Number of qubits
shots = 100

print("Measurement Results:")
for i in range(shots):
    state = initialize_state(N)
    for j in range(N):
        state = H(j, state)
    outcome, state = measure(0, state)
    print(outcome, end='')

    if outcome == 0:
        count_0 += 1
    else:
        count_1 += 1

prob_0 = count_0/shots
prob_1 = count_1/shots

print("\n\nProbability of |0>:", prob_0)
print("\nProbability of |1>:", prob_1)

Measurement Results:
0100111010000011101110000111101111110001001110111100001101000001011001001010110011100001100011101101

Probability of |0>: 0.49

Probability of |1>: 0.51


### 2. Calculating the Exact Expectation Value